In [22]:
#libraries used

import os
import pandas as pd
import numpy as np
import librosa
from pymongo import MongoClient


In [12]:
#.mp3 audio files loaded as it is a sampled 20gb audio dataset
#folder contains only audio file and nothing else


import os
import librosa

def load_audio_files(folder):
    audio_files = []
    for file_name in os.listdir(folder):
        if file_name.endswith('.mp3'):
            file_path = os.path.join(folder, file_name)
            try:
                # Load audio file
                y, sr = librosa.load(file_path, sr=None)
                # Check if audio file is empty or all zero samples
                if len(y) == 0 or y.max() == 0:
                    print(f"Skipping empty or null audio file: {file_name}")
                    continue
                audio_files.append(file_path)
            except Exception as e:
                print(f"Skipping corrupted file: {file_name} ({e})")
    return audio_files

def main():
    # Set the path to the folder containing audio files
    audio_folder = "sample_audio"

    # Load audio files
    audio_files = load_audio_files(audio_folder)

    print(f"Number of valid audio files loaded: {len(audio_files)}")

if __name__ == "__main__":
    main()


Skipping empty or null audio file: 001376.mp3


C:\Users\Tech House\AppData\Local\Temp\ipykernel_19452\3755398217.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=None)


Skipping corrupted file: 001486.mp3 ()
Skipping corrupted file: 002624.mp3 ()
Skipping corrupted file: 003284.mp3 ()
Skipping corrupted file: 005574.mp3 ()
Skipping corrupted file: 008669.mp3 ()
Skipping corrupted file: 010116.mp3 ()
Skipping empty or null audio file: 010256.mp3
Skipping corrupted file: 011583.mp3 ()
Skipping corrupted file: 012838.mp3 ()
Skipping empty or null audio file: 012977.mp3
Skipping corrupted file: 013529.mp3 ()
Skipping corrupted file: 014116.mp3 ()
Skipping corrupted file: 014180.mp3 ()
Skipping corrupted file: 018924.mp3 ()
Skipping empty or null audio file: 020608.mp3
Skipping empty or null audio file: 020609.mp3
Skipping empty or null audio file: 020610.mp3
Skipping empty or null audio file: 020612.mp3
Skipping empty or null audio file: 020614.mp3
Skipping empty or null audio file: 020615.mp3
Skipping empty or null audio file: 020617.mp3
Skipping empty or null audio file: 020618.mp3
Skipping empty or null audio file: 020620.mp3
Skipping empty or null aud

In [21]:
#extracting features of the audio file such as MFCC, spectral centroid and zero crossing rate
#for enchancing the recommendation model of music system

def extract_features(audio_file, n_mfcc=13, hop_length=512, n_fft=2048):
    try:
        # Load audio file
        y, sr = librosa.load(audio_file, sr=None)
        
        # Extract MFCCs
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
        
        # Extract spectral centroid
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
        
        # Extract zero-crossing rate
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y, hop_length=hop_length)
        
        return mfcc, spectral_centroid, zero_crossing_rate
    except Exception as e:
        print(f"Error processing {audio_file}: {e}")
        return None, None, None

def main():
    # Directory containing audio files
    audio_dir = "sample_audio"
    audio_files = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".mp3")]

    for audio_file in audio_files:
        print(f"Extracting features from {audio_file}...")
        mfcc, spectral_centroid, zero_crossing_rate = extract_features(audio_file)
        if mfcc is not None and spectral_centroid is not None and zero_crossing_rate is not None:
            print("MFCC shape:", mfcc.shape)
            print("Spectral centroid shape:", spectral_centroid.shape)
            print("Zero-crossing rate shape:", zero_crossing_rate.shape)
        print()

if __name__ == "__main__":
    main()


Extracting features from sample_audio\000002.mp3...
MFCC shape: (13, 2582)
Spectral centroid shape: (1, 2582)
Zero-crossing rate shape: (1, 2582)

Extracting features from sample_audio\000003.mp3...
MFCC shape: (13, 2585)
Spectral centroid shape: (1, 2585)
Zero-crossing rate shape: (1, 2585)

Extracting features from sample_audio\000005.mp3...
MFCC shape: (13, 2585)
Spectral centroid shape: (1, 2585)
Zero-crossing rate shape: (1, 2585)

Extracting features from sample_audio\000010.mp3...
MFCC shape: (13, 2582)
Spectral centroid shape: (1, 2582)
Zero-crossing rate shape: (1, 2582)

Extracting features from sample_audio\000020.mp3...
MFCC shape: (13, 2582)
Spectral centroid shape: (1, 2582)
Zero-crossing rate shape: (1, 2582)

Extracting features from sample_audio\000026.mp3...
MFCC shape: (13, 2582)
Spectral centroid shape: (1, 2582)
Zero-crossing rate shape: (1, 2582)

Extracting features from sample_audio\000030.mp3...
MFCC shape: (13, 2582)
Spectral centroid shape: (1, 2582)
Zero-cro

KeyboardInterrupt: 

In [23]:
#storing data and its features in mongoDB by assigning a database and collection name

def store_features_in_mongodb(audio_dir, database_name, collection_name):
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    
    # Access or create the database
    db = client[database_name]
    
    # Access or create the collection
    collection = db[collection_name]
    
    # Iterate through audio files
    audio_files = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".mp3")]
    for audio_file in audio_files:
        print(f"Extracting features from {audio_file}...")
        mfcc, spectral_centroid, zero_crossing_rate = extract_features(audio_file)
        if mfcc is not None and spectral_centroid is not None and zero_crossing_rate is not None:
            # Insert features into MongoDB
            document = {
                "audio_file": audio_file,
                "mfcc": mfcc.tolist(),
                "spectral_centroid": spectral_centroid.tolist(),
                "zero_crossing_rate": zero_crossing_rate.tolist()
            }
            collection.insert_one(document)
            print("Features stored in MongoDB successfully.")
        print()

# Example usage
audio_dir = "sample_audio"
database_name = "audio_features"
collection_name = "features"
store_features_in_mongodb(audio_dir, database_name, collection_name)

Extracting features from sample_audio\000002.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000003.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000005.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000010.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000020.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000026.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000030.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000046.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000048.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000134.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000135.mp3...
Features

Features stored in MongoDB successfully.

Extracting features from sample_audio\000237.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000238.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000246.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000247.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000248.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000249.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000250.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000251.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000252.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000253.mp3...
Features stored in MongoDB successfully.

Extracting feature

Features stored in MongoDB successfully.

Extracting features from sample_audio\000364.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000365.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000366.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000367.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000368.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000369.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000370.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000371.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000384.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000385.mp3...
Features stored in MongoDB successfully.

Extracting feature

Features stored in MongoDB successfully.

Extracting features from sample_audio\000478.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000479.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000480.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000481.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000482.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000483.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000484.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000485.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000486.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000487.mp3...
Features stored in MongoDB successfully.

Extracting feature

Features stored in MongoDB successfully.

Extracting features from sample_audio\000575.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000576.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000577.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000578.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000579.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000580.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000583.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000584.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000585.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\000586.mp3...
Features stored in MongoDB successfully.

Extracting feature

...

Features stored in MongoDB successfully.

Extracting features from sample_audio\037299.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037300.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037301.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037302.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037303.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037304.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037305.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037306.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037307.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037308.mp3...
Features stored in MongoDB successfully.

Extracting feature

Features stored in MongoDB successfully.

Extracting features from sample_audio\037455.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037456.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037457.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037458.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037459.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037460.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037461.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037462.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037472.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037473.mp3...
Features stored in MongoDB successfully.

Extracting feature

Features stored in MongoDB successfully.

Extracting features from sample_audio\037582.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037583.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037584.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037585.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037586.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037587.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037588.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037589.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037590.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037591.mp3...
Features stored in MongoDB successfully.

Extracting feature

Features stored in MongoDB successfully.

Extracting features from sample_audio\037716.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037717.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037718.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037719.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037720.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037721.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037722.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037723.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037724.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037725.mp3...
Features stored in MongoDB successfully.

Extracting feature

Features stored in MongoDB successfully.

Extracting features from sample_audio\037902.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037903.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037904.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037905.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037906.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037907.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037908.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037909.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037910.mp3...
Features stored in MongoDB successfully.

Extracting features from sample_audio\037911.mp3...
Features stored in MongoDB successfully.

Extracting feature